### Import Library for the data cleaning and ETL pipleline

In [117]:
# import packages for ETL pipeline
import pandas as pd
import re

In [118]:
# Load disaster category CSV file
categories_df = pd.read_csv('disaster_categories.csv')
categories_df.head(2)

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...


In [119]:
# Load disaster messages CSV file
messages_df = pd.read_csv('disaster_messages.csv')
messages_df.head(2)

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct


In [120]:
# Merge data set from messages and categories
df = messages_df.merge(categories_df, on='id')
df.head(2)

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...


In [121]:
# Found that original columns contains some non field.
df.isnull().mean()

id            0.000000
message       0.000000
original      0.611688
genre         0.000000
categories    0.000000
dtype: float64

In [122]:
# Let's expend the category columns
# create a dataframe of the 36 individual category columns
categories = df['categories'].str.split(';', expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [123]:
# Selected only columns names of the DataFrame
row = categories.iloc[0]
category_colnames = row.str.split('-').str[0]
category_colnames

0                    related
1                    request
2                      offer
3                aid_related
4               medical_help
5           medical_products
6          search_and_rescue
7                   security
8                   military
9                child_alone
10                     water
11                      food
12                   shelter
13                  clothing
14                     money
15            missing_people
16                  refugees
17                     death
18                 other_aid
19    infrastructure_related
20                 transport
21                 buildings
22               electricity
23                     tools
24                 hospitals
25                     shops
26               aid_centers
27      other_infrastructure
28           weather_related
29                    floods
30                     storm
31                      fire
32                earthquake
33                      cold
34            

In [124]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [125]:
# Split and convert to int values
import re
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str.split('-').str[1]
    
    # convert column from string to numeric
    categories[column] = categories[column].astype(int)
    
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
# No null value detect, perfect for next phase 
categories.isnull().mean()
for i in categories.columns.values:
    unique = categories[i].unique()
    if len(unique) == 1: 
        print('** REVIEW ** ', i, ': ', len(unique), ' ', unique)
    else:
        print(i, ': ', len(unique), ' ', unique)
# Data show that related field has 3 unique values [0,1,2] and child_alone only one values
# [0, 1, 2] -> lelated, decide to convert to balance the dataset
# [0] -> child_alone, decide to delete as only have one value

related :  3   [1 0 2]
request :  2   [0 1]
offer :  2   [0 1]
aid_related :  2   [0 1]
medical_help :  2   [0 1]
medical_products :  2   [0 1]
search_and_rescue :  2   [0 1]
security :  2   [0 1]
military :  2   [0 1]
** REVIEW **  child_alone :  1   [0]
water :  2   [0 1]
food :  2   [0 1]
shelter :  2   [0 1]
clothing :  2   [0 1]
money :  2   [0 1]
missing_people :  2   [0 1]
refugees :  2   [0 1]
death :  2   [0 1]
other_aid :  2   [0 1]
infrastructure_related :  2   [0 1]
transport :  2   [0 1]
buildings :  2   [0 1]
electricity :  2   [0 1]
tools :  2   [0 1]
hospitals :  2   [0 1]
shops :  2   [0 1]
aid_centers :  2   [0 1]
other_infrastructure :  2   [0 1]
weather_related :  2   [0 1]
floods :  2   [0 1]
storm :  2   [0 1]
fire :  2   [0 1]
earthquake :  2   [0 1]
cold :  2   [0 1]
other_weather :  2   [0 1]
direct_report :  2   [0 1]


In [127]:
df = pd.concat([df, categories], axis=1, join='inner')
df = df.drop(['categories'], axis=1)

In [128]:
# - Step for clean and balance the database 
# Deleted the child_alone calumns 
df = df.drop(['child_alone'], axis=1)
df["related"] = df["related"].map(lambda x: 1 if x == 2 else x)

In [129]:
print(df.columns)
# Checking the step which we perform 
print(df["related"].unique())

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')
[1 0]


In [130]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [131]:
# check number of duplicates
duplicate = df[df.duplicated()]
duplicate.shape

(171, 39)

In [132]:
# drop duplicates
df = df.drop_duplicates()

In [133]:
# check number of duplicates
duplicate = df[df.duplicated()]
duplicate.shape

(0, 39)

In [134]:
df.shape

(26215, 39)

In [135]:
# Insert into the SQLalchemy 
from sqlalchemy import create_engine
engine = create_engine('sqlite:///disaster_response_table.db')
df.to_sql('disaster_response_table.db', engine, index=False)

#### Final data dump into the disaster_response_table.db with the help of sqliteDB, sqlalchemy engine and pandas DF